### Capturando todos os foros, matérias, unidades, matérias e feitos em andamento do documento
- Versão anterior capturava apenas a primeira ocorrência no PDF
- Acréscimo do gc.collect() a cada iteração

In [ ]:
# import os
# import re
# import gc
# import time
# import numpy as np
# import pandas as pd
# from PyPDF2 import PdfReader
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from natsort import natsorted
# from tqdm import tqdm

# MATERIAS = [
#     "CÍVEL", "CRIMINAL", "EXECUÇÃO FISCAL", "INFÂNCIA",
#     "JUIZADO CRIMINAL", "JUIZADO ESPECIAL",
#     "JUIZADO FAZENDA PÚBLICA", "SETOR FAZENDA PÚBLICA"
# ]

# # --- Extração de texto do PDF ---
# def extrair_texto_pdf(file_path):
#     try:
#         reader = PdfReader(file_path)
#         texto = " ".join(
#             page.extract_text().replace('\n', ' ').strip()
#             for page in reader.pages if page.extract_text()
#         )
#         return re.sub(r"Considerações\s+para\s+análise\s+dos\s+dados.*$", "", texto, flags=re.IGNORECASE)
#     except Exception as e:
#         print(f"❌ Erro ao extrair texto de {file_path}: {e}")
#         return ""

# # --- Extração estruturada por bloco ---
# def extrair_blocos_completo(texto):
#     padrao = (
#         r"Foro:\s*(.+?)\s+Unidade:\s*(.+?)\s+Matéria:\s*"
#         r"(CÍVEL|CRIMINAL|EXECUÇÃO FISCAL|INFÂNCIA|JUIZADO CRIMINAL|"
#         r"JUIZADO ESPECIAL|JUIZADO FAZENDA PÚBLICA|SETOR FAZENDA PÚBLICA)"
#     )
#     matches = list(re.finditer(padrao, texto))
#     blocos = []

#     for i, m in enumerate(matches):
#         start_idx = m.start()
#         end_idx = matches[i + 1].start() if i + 1 < len(matches) else len(texto)
#         bloco = texto[start_idx:end_idx]

#         foro = m.group(1).strip()
#         unidade = m.group(2).strip()
#         materia = m.group(3).strip()

#         feitos = re.search(r"(?i)\bTotal\s+de\s+Feitos\s+em\s+Andamento\s*(\d[\d\.]*)", bloco)
#         feitos_tot = int(feitos.group(1).replace(".", "")) if feitos else np.nan

#         blocos.append({
#             "Foro": foro,
#             "Unidade": unidade,
#             "Matéria": materia,
#             "regex": bloco.strip(),
#             "feitos_tot": feitos_tot
#         })

#     return blocos

# # --- Processamento individual de PDF ---
# def processar_pdf(nome_arquivo, caminho_pdf, safra):
#     texto = extrair_texto_pdf(caminho_pdf)
#     if not texto.strip():
#         print(f"⚠️ Texto vazio para {nome_arquivo}\n")
#         return []

#     blocos = extrair_blocos_completo(texto)
#     if not blocos:
#         print(f"⚠️ Nenhum bloco identificado em {nome_arquivo}\n")

#     for bloco in blocos:
#         bloco["Safra"] = safra
#         bloco["Arquivo"] = nome_arquivo
#     return blocos

# # --- Processamento em lote com paralelismo ---
# def processar_pdfs_em_lote(pasta_base, max_threads=5):
#     if not os.path.exists(pasta_base):
#         raise FileNotFoundError(f"Pasta '{pasta_base}' não encontrada.\n")

#     safras = [s for s in os.listdir(pasta_base) if os.path.isdir(os.path.join(pasta_base, s))]
#     todos_dados = []
#     inicio_total = time.time()

#     for safra in safras:
#         print(f"\n📂 Safra: {safra}")
#         inicio_safra = time.time()
#         pasta_safra = os.path.join(pasta_base, safra)
#         arquivos_pdf = natsorted([f for f in os.listdir(pasta_safra) if f.endswith(".pdf")])
#         resultados = []

#         with ThreadPoolExecutor(max_workers=max_threads) as executor:
#             # 🔁 Modo A — com tqdm simples (sem nome de arquivo)
#             futuros = [
#                 executor.submit(processar_pdf, nome, os.path.join(pasta_safra, nome), safra)
#                 for nome in arquivos_pdf
#             ]

#             for futuro in tqdm(as_completed(futuros), total=len(futuros), desc=f"Processando {safra}"):
#                 resultado = futuro.result()
#                 if resultado:
#                     resultados.extend(resultado)

#             # --- OU, se preferir log detalhado com nome do arquivo:
#             # futuros = {
#             #     executor.submit(processar_pdf, nome, os.path.join(pasta_safra, nome), safra): nome
#             #     for nome in arquivos_pdf
#             # }
#             #
#             # for i, futuro in enumerate(as_completed(futuros), start=1):
#             #     resultado = futuro.result()
#             #     if resultado:
#             #         resultados.extend(resultado)
#             #         nome = futuros[futuro]
#             #         print(f" - [{i}/{len(futuros)}] {nome} da safra {safra} processado.")

#         df_safra = pd.DataFrame(resultados)
#         df_safra["feitos_tot"] = pd.to_numeric(df_safra["feitos_tot"], errors="coerce").astype("Int64")
#         todos_dados.append(df_safra)

#         # 🔁 Liberação de memória
#         del resultados, df_safra
#         gc.collect()

#         print(f"✅ Safra {safra} processada: {len(arquivos_pdf)} arquivos.")
#         print(f"⏱️ Tempo safra: {time.time() - inicio_safra:.1f} segundos")

#     if todos_dados:
#         df_final = pd.concat(todos_dados, ignore_index=True)

#         colunas = ["Foro", "Unidade", "Matéria", "Arquivo", "regex", "Safra", "feitos_tot"]
#         df_final = df_final[colunas]

#         caminho_csv = os.path.join(pasta_base, "foros_unidades_feitos_geral_TESTE.csv")
#         df_final.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
#         print(f"\n🎯 CSV final salvo: {caminho_csv} ({len(df_final)} registros)")
#         print(f"⏳ Tempo total: {time.time() - inicio_total:.1f} segundos")
#     else:
#         print("⚠️ Nenhum dado foi consolidado.\n")

In [ ]:
processar_pdfs_em_lote("teste_1905/", max_threads=4)

### TENTATIVA DE OTIMIZAR TEMPO DE PROCESSAMENTO

In [21]:
import os
import re
import gc
import time
import numpy as np
import pandas as pd
from PyPDF2 import PdfReader
from concurrent.futures import ThreadPoolExecutor, as_completed
from natsort import natsorted
from tqdm import tqdm
import multiprocessing

# --- Constantes ---
MATERIAS = [
    "CÍVEL", "CRIMINAL", "EXECUÇÃO FISCAL", "INFÂNCIA",
    "JUIZADO CRIMINAL", "JUIZADO ESPECIAL",
    "JUIZADO FAZENDA PÚBLICA", "SETOR FAZENDA PÚBLICA"
]

PADRAO_MATERIA = re.compile(
    r"Foro:\s*(.+?)\s+Unidade:\s*(.+?)\s+Matéria:\s*"
    r"(CÍVEL|CRIMINAL|EXECUÇÃO FISCAL|INFÂNCIA|JUIZADO CRIMINAL|"
    r"JUIZADO ESPECIAL|JUIZADO FAZENDA PÚBLICA|SETOR FAZENDA PÚBLICA)",
    flags=re.IGNORECASE
)

PADRAO_FEITOS = re.compile(r"(?i)\bTotal\s+de\s+Feitos\s+em\s+Andamento\s*(\d[\d\.]*)")

# --- Extração de texto do PDF ---
def extrair_texto_pdf(file_path):
    try:
        reader = PdfReader(file_path)
        texto = " ".join(
            t.replace('\n', ' ').strip()
            for page in reader.pages
            for t in [page.extract_text()]
            if t
        )
        return re.sub(r"Considerações\s+para\s+análise\s+dos\s+dados.*$", "", texto, flags=re.IGNORECASE)
    except Exception as e:
        print(f"❌ Erro ao extrair texto de {file_path}: {e}")
        return ""

# --- Extração estruturada por bloco ---
def extrair_blocos_completo(texto):
    matches = list(PADRAO_MATERIA.finditer(texto))
    blocos = []
    for i, m in enumerate(matches):
        start_idx = m.start()
        end_idx = matches[i + 1].start() if i + 1 < len(matches) else len(texto)
        bloco = texto[start_idx:end_idx]

        feitos = PADRAO_FEITOS.search(bloco)
        feitos_tot = int(feitos.group(1).replace(".", "")) if feitos else np.nan

        blocos.append({
            "Foro": m.group(1).strip(),
            "Unidade": m.group(2).strip(),
            "Matéria": m.group(3).strip(),
            "regex": bloco.strip(),
            "feitos_tot": feitos_tot
        })

    return blocos

# --- Processamento individual de PDF ---
def processar_pdf(nome_arquivo, caminho_pdf, safra):
    texto = extrair_texto_pdf(caminho_pdf)
    if not texto.strip():
        print(f"⚠️ Texto vazio para {nome_arquivo}\n")
        return []

    blocos = extrair_blocos_completo(texto)
    if not blocos:
        print(f"⚠️ Nenhum bloco identificado em {nome_arquivo}\n")

    for bloco in blocos:
        bloco["Safra"] = safra
        bloco["Arquivo"] = nome_arquivo
    return blocos

# --- Processamento em lote com paralelismo ---
def processar_pdfs_em_lote(pasta_base, max_threads=None):
    if not os.path.exists(pasta_base):
        raise FileNotFoundError(f"Pasta '{pasta_base}' não encontrada.\n")

    if max_threads is None:
        max_threads = max(1, multiprocessing.cpu_count() - 1)

    safras = [s for s in os.listdir(pasta_base) if os.path.isdir(os.path.join(pasta_base, s))]
    todos_dados = []
    inicio_total = time.time()

    for safra in safras:
        print(f"\n📂 Safra: {safra}")
        inicio_safra = time.time()
        pasta_safra = os.path.join(pasta_base, safra)
        arquivos_pdf = natsorted([f for f in os.listdir(pasta_safra) if f.endswith(".pdf")])
        resultados = []

        with ThreadPoolExecutor(max_workers=max_threads) as executor:
            futuros = [
                executor.submit(processar_pdf, nome, os.path.join(pasta_safra, nome), safra)
                for nome in arquivos_pdf
            ]

            for futuro in tqdm(as_completed(futuros), total=len(futuros), desc=f"Processando {safra}"):
                resultado = futuro.result()
                if resultado:
                    resultados.extend(resultado)

        df_safra = pd.DataFrame(resultados)
        df_safra["feitos_tot"] = pd.to_numeric(df_safra["feitos_tot"], errors="coerce").astype("Int64")
        todos_dados.append(df_safra)

        del resultados, df_safra
        gc.collect()

        print(f"✅ Safra {safra} processada: {len(arquivos_pdf)} arquivos.")
        print(f"⏱️ Tempo safra: {time.time() - inicio_safra:.1f} segundos")

    if todos_dados:
        df_final = pd.concat(todos_dados, ignore_index=True)
        colunas = ["Foro", "Unidade", "Matéria", "Arquivo", "regex", "Safra", "feitos_tot"]
        df_final = df_final[colunas]

        caminho_csv = os.path.join(pasta_base, "foros_unidades_feitos_geral_PY.csv")
        df_final.to_csv(caminho_csv, index=False, encoding="utf-8-sig")
        print(f"\n🎯 CSV final salvo: {caminho_csv} ({len(df_final)} registros)")
        print(f"⏳ Tempo total: {time.time() - inicio_total:.1f} segundos")
    else:
        print("⚠️ Nenhum dado foi consolidado.\n")


In [ ]:
processar_pdfs_em_lote("demanda_mari/")

In [3]:
df = pd.read_csv("teste_1905/foros_unidades_feitos_geral_PY.csv")

print(df['feitos_tot'].sum())

33706749.0


In [14]:
texto = extrair_texto_pdf('./teste_1905/202412/unidade_100.pdf')

texto

'CORREGEDORIA GERAL DA JUSTIÇA Controle do Movimento Judiciário. Produtividade  1 ª Instância. Foro:  CENTRAL Unidade:  27 CRIMINAL Matéria:  CRIMINAL Feitos em andamento 12/2024 Procedimentos Investigatórios 197 Processo crime e queixas 3.141 Cartas Precatórias, Rogatórias ou de  Ordem 75 Outros Procedimentos 10 Feitos em grau de recurso 228 Total de feitos em andamento 3.455 Entradas no mês 12/2024 Distribuídos 16 Entradas por redistribuição 69 Desarquivados com reabertura 1 Recebidos dos tribunais 20 Denúncias Recebidas 52 Total 158 Saídas no mês 12/2024 Arquivados definitivamente 134 Arquivo Provisório 4 Saídas por redistribuição 7 Enviados aos tribunais 32 Precatórias devolvidas 24 Total 201 Produtividade de magistrados(as) 12/2024 Pauta titular 5 Audiências realizadas 87 Sentenças proferidas 76 Decisão interlocutória 452 Despacho 449 Conclusos para Sentença 1 Página  1 Consulta realizada em: 21/05/2025 11:48:45 CORREGEDORIA GERAL DA JUSTIÇA Controle do Movimento Judiciário. Produ

In [15]:
extrair_blocos_completo(texto)

[{'Foro': 'CENTRAL',
  'Unidade': '27 CRIMINAL',
  'Matéria': 'CRIMINAL',
  'regex': 'Foro:  CENTRAL Unidade:  27 CRIMINAL Matéria:  CRIMINAL Feitos em andamento 12/2024 Procedimentos Investigatórios 197 Processo crime e queixas 3.141 Cartas Precatórias, Rogatórias ou de  Ordem 75 Outros Procedimentos 10 Feitos em grau de recurso 228 Total de feitos em andamento 3.455 Entradas no mês 12/2024 Distribuídos 16 Entradas por redistribuição 69 Desarquivados com reabertura 1 Recebidos dos tribunais 20 Denúncias Recebidas 52 Total 158 Saídas no mês 12/2024 Arquivados definitivamente 134 Arquivo Provisório 4 Saídas por redistribuição 7 Enviados aos tribunais 32 Precatórias devolvidas 24 Total 201 Produtividade de magistrados(as) 12/2024 Pauta titular 5 Audiências realizadas 87 Sentenças proferidas 76 Decisão interlocutória 452 Despacho 449 Conclusos para Sentença 1 Página  1 Consulta realizada em: 21/05/2025 11:48:45 CORREGEDORIA GERAL DA JUSTIÇA Controle do Movimento Judiciário. Produtividade 